In [36]:
# 1. Install libraries if not already installed
!pip install pymongo mysql-connector-python pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 37.7 MB/s eta 0:00:00a 0:00:01


In [37]:
# 2. Import libraries
import mysql.connector
from pymongo import MongoClient
import pandas as pd
from datetime import datetime
from decimal import Decimal

In [ ]:
# 3. MySQL Connection
mysql_conn = mysql.connector.connect(
    host='mdsmysql.sci.pitt.edu',
    user='mdsGlobalUser',
    password='mds$uper$ecurePassword123',
    database='sakila'
)
mysql_cursor = mysql_conn.cursor(dictionary=True)

In [39]:
# 4. MongoDB Connection
username = "etladmin"
password = "securePassword123"
host = "mdsmongodb.sci.pitt.edu"
port = 27017
auth_db = "sakila"  # or the database where the user is defined

# Format the connection string
uri = f"mongodb://{username}:{password}@{host}:{port}/?authSource={auth_db}"

mongo_client = MongoClient(uri)
mongo_db = mongo_client['sakila']
mongo_client.admin.command('ping')  # Trigger connection
print("✅ Successfully connected to MongoDB with authentication.")
print("Available databases:", mongo_client.list_database_names())

✅ Successfully connected to MongoDB with authentication.
Available databases: []


In [40]:
# 5. List all tables in Sakila MySQL
mysql_cursor.execute("SHOW TABLES")
tables = [row[f'Tables_in_sakila'] for row in mysql_cursor.fetchall()]

print(f"Found tables: {tables}")


Found tables: ['actor', 'actor_info', 'address', 'category', 'city', 'country', 'customer', 'customer_list', 'film', 'film_actor', 'film_category', 'film_list', 'film_text', 'inventory', 'language', 'nicer_but_slower_film_list', 'payment', 'rental', 'sales_by_film_category', 'sales_by_store', 'staff', 'staff_list', 'store']


In [43]:

# 6. Define a transformation function
def transform_types(record):
    transformed = {}
    for key, value in record.items():
        if isinstance(value, Decimal):
            transformed[key] = float(value)
        elif isinstance(value, datetime):
            transformed[key] = value  # Already compatible
        elif isinstance(value, set):
            transformed[key] = list(value)  # 🔥 Convert sets to lists
        else:
            transformed[key] = value
    return transformed


In [44]:
# 7. Migrate each table
for table in tables:
    print(f"Processing table: {table}")

    mysql_cursor.execute(f"SELECT * FROM {table}")
    rows = mysql_cursor.fetchall()

    # Transform records
    documents = [transform_types(row) for row in rows]

    if documents:
        mongo_collection = mongo_db[table]
        mongo_collection.insert_many(documents)
        print(f"Inserted {len(documents)} records into {table} collection.")
    else:
        print(f"No records found in {table}.")




Processing table: actor
Inserted 200 records into actor collection.
Processing table: actor_info
Inserted 200 records into actor_info collection.
Processing table: address
Inserted 603 records into address collection.
Processing table: category
Inserted 16 records into category collection.
Processing table: city
Inserted 600 records into city collection.
Processing table: country
Inserted 109 records into country collection.
Processing table: customer
Inserted 599 records into customer collection.
Processing table: customer_list
Inserted 599 records into customer_list collection.
Processing table: film
Inserted 1000 records into film collection.
Processing table: film_actor
Inserted 5462 records into film_actor collection.
Processing table: film_category
Inserted 1000 records into film_category collection.
Processing table: film_list
Inserted 1000 records into film_list collection.
Processing table: film_text
Inserted 1000 records into film_text collection.
Processing table: inventory


In [ ]:
# 8. Clean up
mysql_cursor.close()
mysql_conn.close()
mongo_client.close()

print("✅ Migration complete!")